# Prepare data tables for DNA counting and other information that will be used to generate figures

In [1]:
#import libraries
import pandas as pd
import os
import bioframe as bf
import numpy as np
import warnings
warnings.filterwarnings('ignore')
os.chdir('..\data')

In [2]:
#import key tables
#this is the original table with seq information
df = pd.read_csv('p63enh_starrseq_info.csv', usecols=['uniqueID', 'obs_score','p63RE_class',
													  'WT', 'mut', 'shuffle','flankShuffle','fullShuffle',
													  'strand','p63RE_chrom','p63RE_start','p63RE_stop'])
#make a long table and reformat
df_long = df.melt(id_vars=['uniqueID', 'obs_score','p63RE_class','strand','p63RE_chrom','p63RE_start','p63RE_stop'],
				  value_vars=['WT', 'mut', 'shuffle','flankShuffle', 'fullShuffle'],
				  var_name='enhancer_variant',
				  value_name='seq')
#add unique id with p63RE locationa and variant
df_long['id'] = df_long.apply(lambda row: row.uniqueID[: row.uniqueID.rfind('_') + 1] + row.enhancer_variant, axis='columns')
df_long['loc'] = df_long.apply(lambda row: row.uniqueID[: row.uniqueID.rfind('_')], axis='columns')
df_long[['p63RE_class', 'p63RE_type']] = df_long.apply(lambda row: [row.p63RE_class[:row.p63RE_class.find('.')],
																	row.p63RE_class[row.p63RE_class.find('.') + 1:]],
													    axis='columns', result_type='expand')
df_long.replace({'unique-p63RE':'Unique p63RE', 'p53+p63RE' : 'p53RE+p63RE'}, inplace=True)
df_long.drop(columns=['uniqueID', ], inplace=True)
#add enhancner GC content (%)
def gc_count(sequence):
	sequence = sequence.upper()
	gc = ((sequence.count('G') + sequence.count('C')) / len(sequence)) * 100
	return gc
df_long['gc'] = df_long.seq.apply(lambda x: gc_count(x))
riege_xls = pd.ExcelFile('elife-63266-supp3-v2.xlsx', engine='openpyxl')
p53_motifs = riege_xls.parse('p53peaks+REs', usecols=['Number of supporting p53 ChIP-seq datasets','p53RE_Chromosome','p53RE_Start','p53RE_End'])
#remove rows with no motif coordinates
p53_motifs = p53_motifs[p53_motifs.p53RE_Start != '-']
#bioframe needs int
p53_motifs =p53_motifs.astype({"p53RE_Start": int, "p53RE_End": int})
p53_p63_overlap = bf.overlap(df_long, p53_motifs,  cols1 = ['p63RE_chrom','p63RE_start','p63RE_stop'],
							  cols2 = ['p53RE_Chromosome','p53RE_Start','p53RE_End'], suffixes = ['','_p53'])
del(riege_xls,p53_motifs)
#save enh id that has p53 ob if they overlap with p63RE
df_long = df_long.merge(p53_p63_overlap[['id','Number of supporting p53 ChIP-seq datasets_p53']], on='id', how='left')
#df_long.to_csv('p63enh_starrseq_info_long.csv')
df_long.sample(10)

,obs_score,p63RE_class,strand,p63RE_chrom,p63RE_start,p63RE_stop,enhancer_variant,seq,id,loc,p63RE_type,gc,Number of supporting p53 ChIP-seq datasets_p53
81004,9,primary,-,chr12,3120237,3120261,fullShuffle,cgacccacaaacggtgcacctcggaaggaattggagattatacgcg...,chr12_3120237_3120261_fullShuffle,chr12_3120237_3120261,Unique p63RE,53.333333,NaN
68466,8,primary,+,chr5,112217883,112217902,flankShuffle,tggtgtagttggaggtggggacagaagaaggatgctgaagagatga...,chr5_112217883_112217902_flankShuffle,chr5_112217883_112217902,p53RE+p63RE,48.739496,5.0
35965,18,primary,+,chr7,56009912,56009931,shuffle,acaccaggcggtgcagaggctgatgccgtttgcagcacttttcaag...,chr7_56009912_56009931_shuffle,chr7_56009912_56009931,p53RE+p63RE,53.781513,8.0
73028,15,primary,+,chr4,112277408,112277432,fullShuffle,aagtatgcagtctaaaaggacatacacccaccgacatcaggaaagt...,chr4_112277408_112277432_fullShuffle,chr4_112277408_112277432,Unique p63RE,49.166667,NaN
61976,10,primary,+,chr15,65979638,65979662,flankShuffle,gaagtgcggcgcacttcaagttatatcgcgcgttggttcgcgcaac...,chr15_65979638_65979662_flankShuffle,chr15_65979638_65979662,Unique p63RE,57.500000,NaN
54979,16,primary,-,chr8,119683732,119683751,flankShuffle,tcgtatcatggccgatggaggccagcgaagttggggtatgacgtta...,chr8_119683732_119683751_flankShuffle,chr8_119683732_119683751,p53RE+p63RE,47.058824,NaN
64102,9,primary,-,chr16,67409504,67409523,flankShuffle,cagtgacagacgtgcctttgggcacctcaaccaccagctgcgagag...,chr16_67409504_67409523_flankShuffle,chr16_67409504_67409523,p53RE+p63RE,57.142857,7.0
59707,12,secondary,+,chr7,30166638,30166657,flankShuffle,accgggacgtcccacaaggacagagatggagcacttttagggggtt...,chr7_30166638_30166657_flankShuffle,chr7_30166638_30166657,Unique p63RE,54.621849,NaN
36143,17,primary,-,chr10,71889165,71889189,shuffle,actcgtggcctgacgcatccgcgagactgcacgcggccaggcgggg...,chr10_71889165_71889189_shuffle,chr10_71889165_71889189,Unique p63RE,73.333333,9.0
84807,8,primary,+,chr2,108406021,108406040,fullShuffle,taatagtgtgttagataacacaaatgcagctcgacataattgtaca...,chr2_108406021_108406040_fullShuffle,chr2_108406021_108406040,p53RE+p63RE,41.176471,NaN


# Prepare raw counts table from all p63 enhancer STARRseq experiments

In [3]:
#import key tables
os.chdir('..\data')
counts_path = 'read_counts'
#this is the original table with seq information
df = pd.read_csv('p63enh_starrseq_info_long.csv', usecols=['seq', 'id'])
df[['loc','enhancer_variant']] = df.apply(lambda row: [row.id[:row.id.rfind('_')],
													   row.id[row.id.rfind('_') + 1 :]],
										  				axis='columns', result_type='expand')
df.seq = df.seq.apply(lambda x: x[:101].upper())
#gather all the counts tables for each sample
count_tbl = os.listdir(counts_path)
for tbl in count_tbl:
	counts = pd.read_csv(os.path.join(counts_path,tbl), names=['seq',tbl[:tbl.find('.csv')]])
	df = df.merge(counts, how='left', on='seq')
#drop poor quality replicate
df.drop(columns=['SCC25_1_count'], inplace=True)
#df.to_csv('p63enh_starrseq_raw_counts.csv')
df.sample(5)

,seq,id,loc,enhancer_variant,HaCaT_1_count,HaCaT_2_count,MCF10AGus_count,MCF10Ap53KO_1_count,MCF10Ap53KO_2_count,MCF10ATAp63B_count,MCF10A_1_count,MCF10A_2_count,plasmid_1_count,plasmid_2_count,SCC25_2_count,SCC25_4_count
68122,TCTGGACCGCCACCCCTAGACTCTTGAGGGTGATGGCACTTTCAGA...,chr4_7430567_7430586_flankShuffle,chr4_7430567_7430586,flankShuffle,3053,1700,1605,4441,2828,2031,5129,5290,6381,2232,2332,2536
31469,CTATAAAAGTCTCATATCAGTTTTCATCAGCAGATGAAGAGTTACA...,chr10_21458596_21458615_mut,chr10_21458596_21458615,mut,368,323,167,499,212,76,1259,689,2203,805,329,1411
64209,CGTGACGGGAGGATGCCTGATCAACGTGAACGAAGTGCAAAGGCTT...,chr17_74599894_74599913_flankShuffle,chr17_74599894_74599913,flankShuffle,1278,667,2055,1910,2840,3631,3341,4225,1782,571,681,1364
8974,GGAGAGTGTATTTCTGCTACCTACTGGATGATAAAGAAATTTGCTG...,chr3_98339217_98339241_WT,chr3_98339217_98339241,WT,768,537,523,819,1472,1038,1818,1348,1588,510,954,796
45636,CACCCATACAATGGAATATTACTCAGCAATAAGGGTTAATGAAGAA...,chr6_145713916_145713940_shuffle,chr6_145713916_145713940,shuffle,242,329,445,286,205,160,720,717,1205,475,240,194


## Process reads
 - Add counts per million (CPM) 
 - normalize RNA reads to DNA reads (RNA/DNA) = expression
 - All cell types except HaCaT were transfected with plasmid pool 1 so they are normalized to plasmid 1, HaCaTs normalized to plasmid 2 DNA counts.

In [7]:
#import key tables
os.chdir('..\data')
min_reads = 0
#this is the original table with seq information
df = pd.read_csv('p63enh_starrseq_raw_counts.csv', index_col=0)
df.drop(columns=['seq'], inplace=True)
count = len(df)
#get total read counts per pool
total_counts = {'MCF10A_1_count':df.MCF10A_1_count.sum(), 'MCF10A_2_count':df.MCF10A_2_count.sum(),
		 'MCF10Ap53KO_1_count':df.MCF10Ap53KO_1_count.sum(),'MCF10Ap53KO_2_count':df.MCF10Ap53KO_2_count.sum(),
		 'HaCaT_1_count':df.HaCaT_1_count.sum(),'HaCaT_2_count':df.HaCaT_2_count.sum(),
		 'SCC25_2_count':df.SCC25_2_count.sum(),'SCC25_4_count':df.SCC25_4_count.sum(),
		 'plasmid_1_count':df.plasmid_1_count.sum(),'plasmid_2_count':df.plasmid_2_count.sum(),
		 'MCF10AGus_count':df.MCF10AGus_count.sum(), 'MCF10ATAp63B_count':df.MCF10ATAp63B_count.sum()}
#organize and name all the columns
col_list = {'cpm_col':['CPM_RNA_MCF10A_1','CPM_RNA_MCF10A_2','CPM_RNA_MCF10Ap53KO_1','CPM_RNA_MCF10Ap53KO_2','CPM_RNA_MCF10AGus','CPM_RNA_MCF10ATAp63B',
					    'CPM_RNA_SCC25_2','CPM_RNA_SCC25_4','CPM_RNA_HaCaT_1','CPM_RNA_HaCaT_2','CPM_plasmid_1','CPM_plasmid_2'],
			'count_col':['MCF10A_1_count','MCF10A_2_count','MCF10Ap53KO_1_count','MCF10Ap53KO_2_count','MCF10AGus_count','MCF10ATAp63B_count',
				 		 'SCC25_2_count','SCC25_4_count','HaCaT_1_count','HaCaT_2_count','plasmid_1_count','plasmid_2_count'],
			'norm_col':['RNA/DNA_MCF10A_1','RNA/DNA_MCF10A_2','RNA/DNA_MCF10Ap53KO_1','RNA/DNA_MCF10Ap53KO_2','RNA/DNA_MCF10AGus','RNA/DNA_MCF10ATAp63B',
					    'RNA/DNA_SCC25_2','RNA/DNA_SCC25_4','RNA/DNA_HaCaT_1','RNA/DNA_HaCaT_2'],}
#remove rows where where plasmid count was < threshold
df = df[(df.plasmid_1_count > min_reads) & (df.plasmid_2_count > min_reads)]
#normalize to total reads and get CPM
for n in range(0,len(col_list['cpm_col'])):
	df[col_list['cpm_col'][n]] = df[col_list['count_col'][n]].div(total_counts[col_list['count_col'][n]]) * 1000000
#normalize all MCFs and SCCs to plasmid pool 1
df[col_list['norm_col'][0:8]] = df[col_list['cpm_col'][0:8]].div(df['CPM_plasmid_1'].values, axis='index')
#fresh pool of DNA was maxipreped for transfection into HaCats, use pool 2
df[col_list['norm_col'][8:10]] = df[col_list['cpm_col'][8:10]].div(df['CPM_plasmid_2'].values, axis='index')
print('Number of enhancers with at least 1 DNA read:', len(df))
print('Lost enhancers:', count - len(df))
#average RNA/DNA column for 2 replicates
df.drop(columns=['MCF10A_1_count','MCF10A_2_count','MCF10Ap53KO_1_count','MCF10Ap53KO_2_count','MCF10AGus_count','MCF10ATAp63B_count',
				 		 'SCC25_2_count','SCC25_4_count','HaCaT_1_count','HaCaT_2_count','plasmid_1_count','plasmid_2_count'], inplace=True)
df['mean_RNA/DNA_MCF10Ap53KO'] = df[['RNA/DNA_MCF10Ap53KO_1','RNA/DNA_MCF10Ap53KO_2']].mean(axis=1)
df['mean_RNA/DNA_MCF10A'] = df[['RNA/DNA_MCF10A_1','RNA/DNA_MCF10A_2']].mean(axis=1)
df['mean_RNA/DNA_HaCaT'] = df[['RNA/DNA_HaCaT_1','RNA/DNA_HaCaT_2']].mean(axis=1)
df['mean_RNA/DNA_SCC25'] = df[['RNA/DNA_SCC25_4','RNA/DNA_SCC25_2']].mean(axis=1)
#df.to_csv('p63enh_starrseq_norm_counts_unfiltered.csv')
df

Number of enhancers with at least 1 DNA read: 84998
Lost enhancers: 1552


,id,loc,enhancer_variant,CPM_RNA_MCF10A_1,CPM_RNA_MCF10A_2,CPM_RNA_MCF10Ap53KO_1,CPM_RNA_MCF10Ap53KO_2,CPM_RNA_MCF10AGus,CPM_RNA_MCF10ATAp63B,CPM_RNA_SCC25_2,...,RNA/DNA_MCF10AGus,RNA/DNA_MCF10ATAp63B,RNA/DNA_SCC25_2,RNA/DNA_SCC25_4,RNA/DNA_HaCaT_1,RNA/DNA_HaCaT_2,mean_RNA/DNA_MCF10Ap53KO,mean_RNA/DNA_MCF10A,mean_RNA/DNA_HaCaT,mean_RNA/DNA_SCC25
0,chr1_3717092_3717116_WT,chr1_3717092_3717116,WT,12.902554,8.788156,10.014773,8.162671,8.713727,27.701572,6.142934,...,0.565690,1.798371,0.398796,0.925611,0.974044,1.129702,0.590035,0.704075,1.051873,0.662203
1,chr1_5652400_5652424_WT,chr1_5652400_5652424,WT,5.118636,4.249134,4.513400,7.805911,4.374719,13.997650,5.790076,...,1.387837,4.440618,1.836845,1.099817,1.303404,1.092689,1.954091,1.485917,1.198047,1.468331
2,chr1_23167597_23167616_WT,chr1_23167597_23167616,WT,7.342099,8.708056,9.929481,5.793784,6.838847,19.446807,10.706028,...,1.578312,4.488055,2.470804,3.612388,1.112279,1.377355,1.814356,1.852077,1.244817,3.041596
3,chr1_31575754_31575773_WT,chr1_31575754_31575773,WT,10.215719,9.226801,10.448344,14.919708,7.579871,5.449157,11.074924,...,1.236709,0.889068,1.806952,1.327622,1.003116,1.086984,2.069488,1.586092,1.045050,1.567287
4,chr1_38027284_38027308_WT,chr1_38027284_38027308,WT,5.549679,4.920452,8.266275,7.862993,4.571135,12.731582,6.824591,...,1.133709,3.157620,1.692599,1.859988,1.809575,1.941043,2.000148,1.298373,1.875309,1.776294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86545,chr9_135951083_135951107_fullShuffle,chr9_135951083_135951107,fullShuffle,15.718702,13.315735,13.334077,12.543686,9.999359,8.902992,23.080111,...,0.643351,0.572811,1.484956,1.209101,0.808896,0.945708,0.832477,0.934026,0.877302,1.347028
86546,chr9_136484776_136484795_fullShuffle,chr9_136484776_136484795,fullShuffle,1.483506,0.106801,0.902680,0.014270,0.000000,0.840669,0.000000,...,0.000000,2.269281,0.000000,0.000000,0.000000,2.317598,1.237596,2.146417,1.158799,0.000000
86547,chr9_136533879_136533893_fullShuffle,chr9_136533879_136533893,fullShuffle,19.889043,19.704693,24.258639,25.187264,12.927742,23.903367,27.627165,...,0.706899,1.307054,1.510675,1.274251,1.366070,1.237290,1.351870,1.082508,1.301680,1.392463
86548,chr9_137062571_137062595_fullShuffle,chr9_137062571_137062595,fullShuffle,34.673818,43.387706,59.349438,56.368098,41.434843,37.019834,52.343253,...,1.154668,1.031635,1.458654,1.754311,1.138603,1.086019,1.612355,1.087673,1.112311,1.606482


## Read filtering
 - Used minimal CPM cutoff of 2 (DNA) and 0.1 (RNA) for MCF10A WT and p53KO, HaCaT, and MCF10A + GUS or TAp63β

In [15]:
#set CPM cutofffs
min_cpm_2, min_rna_cpm = 2,0.1
df = pd.read_csv('p63enh_starrseq_norm_counts_unfiltered.csv', index_col=0)
#separate tables by cell line for filtering and enhacner matching
mcf = df[['loc','id','enhancer_variant','CPM_RNA_MCF10A_1','CPM_RNA_MCF10A_2',
		  'RNA/DNA_MCF10A_1', 'RNA/DNA_MCF10A_2', 'mean_RNA/DNA_MCF10A','CPM_plasmid_1']].copy()
pko = df[['loc','id','enhancer_variant','CPM_RNA_MCF10Ap53KO_1','CPM_RNA_MCF10Ap53KO_2',
		  'RNA/DNA_MCF10Ap53KO_1', 'RNA/DNA_MCF10Ap53KO_2', 'mean_RNA/DNA_MCF10Ap53KO','CPM_plasmid_1']].copy()
hac = df[['loc','id','enhancer_variant','CPM_RNA_HaCaT_1','CPM_RNA_HaCaT_2',
		  'RNA/DNA_HaCaT_1', 'RNA/DNA_HaCaT_2', 'mean_RNA/DNA_HaCaT','CPM_plasmid_2']].copy()
scc = df[['loc','id','enhancer_variant','CPM_RNA_SCC25_4','CPM_RNA_SCC25_2',
		  'RNA/DNA_SCC25_4', 'RNA/DNA_SCC25_2', 'mean_RNA/DNA_SCC25','CPM_plasmid_1']].copy()
tap63 = df[['loc','id','enhancer_variant','CPM_RNA_MCF10AGus','CPM_RNA_MCF10ATAp63B',
			'RNA/DNA_MCF10AGus','RNA/DNA_MCF10ATAp63B','CPM_plasmid_1']].copy()
mcf = mcf[(mcf.CPM_RNA_MCF10A_1 > min_rna_cpm) & (mcf.CPM_RNA_MCF10A_2 > min_rna_cpm) & (mcf.CPM_plasmid_1 > min_cpm_2)]
pko = pko[(pko.CPM_RNA_MCF10Ap53KO_1 > min_rna_cpm) & (pko.CPM_RNA_MCF10Ap53KO_2 > min_rna_cpm) & (pko.CPM_plasmid_1 > min_cpm_2)]
hac = hac[(hac.CPM_RNA_HaCaT_1 > min_rna_cpm) & (hac.CPM_RNA_HaCaT_2 > min_rna_cpm) & (hac.CPM_plasmid_2 > min_cpm_2)]
scc = scc[(scc.CPM_RNA_SCC25_4 > min_rna_cpm) & (scc.CPM_RNA_SCC25_2 > min_rna_cpm) & (scc.CPM_plasmid_1 > min_cpm_2)]
tap63 = tap63[(tap63.CPM_RNA_MCF10AGus > min_rna_cpm) & (tap63.CPM_RNA_MCF10ATAp63B > min_rna_cpm) & (tap63.CPM_plasmid_1 > min_cpm_2)]
del(df)

In [14]:
tap63

,loc,id,enhancer_variant,CPM_RNA_MCF10AGus,CPM_RNA_MCF10ATAp63B,RNA/DNA_MCF10AGus,RNA/DNA_MCF10ATAp63B,CPM_plasmid_1
0,chr1_3717092_3717116,chr1_3717092_3717116_WT,WT,8.713727,27.701572,0.565690,1.798371,15.403700
1,chr1_5652400_5652424,chr1_5652400_5652424_WT,WT,4.374719,13.997650,1.387837,4.440618,3.152185
2,chr1_23167597_23167616,chr1_23167597_23167616_WT,WT,6.838847,19.446807,1.578312,4.488055,4.333014
3,chr1_31575754_31575773,chr1_31575754_31575773_WT,WT,7.579871,5.449157,1.236709,0.889068,6.129065
4,chr1_38027284_38027308,chr1_38027284_38027308_WT,WT,4.571135,12.731582,1.133709,3.157620,4.032019
...,...,...,...,...,...,...,...,...
86543,chr9_135016437_135016456,chr9_135016437_135016456_fullShuffle,fullShuffle,31.926524,29.686767,0.983829,0.914810,32.451300
86545,chr9_135951083_135951107,chr9_135951083_135951107_fullShuffle,fullShuffle,9.999359,8.902992,0.643351,0.572811,15.542622
86547,chr9_136533879_136533893,chr9_136533879_136533893_fullShuffle,fullShuffle,12.927742,23.903367,0.706899,1.307054,18.287966
86548,chr9_137062571_137062595,chr9_137062571_137062595_fullShuffle,fullShuffle,41.434843,37.019834,1.154668,1.031635,35.884635


## Match enhancer variants and cell types
- Not all enhancers may have a matching variant
- Only use enhancers that have 2 replicates meting CPM cutoff
- Match MCF10A WT and p53KO for all 5 variants (WT, mut, shuffle, flankShuffle, fullShuffle)
- Separate MCF10A p53KO, HaCaT and SCC25, and match for WT and mut variants only
- Separately MCF10A Gus and TAp6B for WT and mut variants only

In [16]:
#merge appropriate tables together
#MCF10A WT and p53KO table
norm_mcf = mcf.merge(pko.drop(columns=['loc','enhancer_variant','CPM_plasmid_1']), on='id', how='inner')
#MCF10A p53KO, SCC25 and HaCaT table (for fig. 5)
norm_mh = pko.merge(hac.drop(columns=['loc','enhancer_variant']), on='id', how='left')
norm_mhs = norm_mh.merge(scc.drop(columns=['loc','enhancer_variant','CPM_plasmid_1']), on='id', how='inner')
#move enh variants into columns to make it easier to match enhancers in each sample pool
#MCF10A WT and KO, all variants
wide_mcf = norm_mcf.pivot(index=['loc'], columns=['enhancer_variant'],
		 values=['RNA/DNA_MCF10A_1','RNA/DNA_MCF10A_2',
		   'RNA/DNA_MCF10Ap53KO_1','RNA/DNA_MCF10Ap53KO_2', 
		   'mean_RNA/DNA_MCF10A', 'mean_RNA/DNA_MCF10Ap53KO']).reset_index()
#MCF10A p53KO, HaCaT, SCC25, WT and mut variants
wide_mhs = norm_mhs[norm_mhs.enhancer_variant.isin(['WT','mut'])].pivot(index=['loc'], columns=['enhancer_variant'],
		 values=['RNA/DNA_MCF10Ap53KO_1','RNA/DNA_MCF10Ap53KO_2', 'mean_RNA/DNA_MCF10Ap53KO',
		   'RNA/DNA_HaCaT_1','RNA/DNA_HaCaT_2', 'mean_RNA/DNA_HaCaT',
		   'RNA/DNA_SCC25_4','RNA/DNA_SCC25_2', 'mean_RNA/DNA_SCC25']).reset_index()
#MCF10A Gus and TAp63B overexpression, WT and mut variants
wide_tap63 = tap63[tap63.enhancer_variant.isin(['WT','mut'])].pivot(index=['loc'], columns=['enhancer_variant'],
		 values=['RNA/DNA_MCF10AGus','RNA/DNA_MCF10ATAp63B']).reset_index()
#drop NaNs in each row to remove enhancers that don't have a match in replicate/cell type/enh variant
wide_mcf.dropna(how='any', inplace=True)
wide_mhs.dropna(how='any', inplace=True)
wide_tap63.dropna(how='any', inplace=True)
print('Number of unique enhancer locations in MCF10As WT and p53KO:', wide_mcf['loc',''].nunique())
print('Number of unique enhancer locations in MCF10As p53KO, HaCaT and SCC25:', wide_mhs['loc',''].nunique())
print('Number of unique enhancer locations in MCF10As Gus and TAp63B:', wide_tap63['loc',''].nunique())
wide_mhs.head(5)

Number of unique enhancer locations in MCF10As WT and p53KO: 10129
Number of unique enhancer locations in MCF10As p53KO, HaCaT and SCC25: 9697
Number of unique enhancer locations in MCF10As Gus and TAp63B: 13532


loc RNA/DNA_MCF10Ap53KO_1            \
enhancer_variant                                               WT       mut   
0                 chr10_100043539_100043558              1.319655  0.560156   
1                 chr10_100157341_100157365              1.671221  1.560248   
2                 chr10_100286133_100286152              0.559138  0.383291   
3                 chr10_100373316_100373330              3.184686  1.870014   
6                 chr10_101013661_101013685              0.471391  0.684964   

                 RNA/DNA_MCF10Ap53KO_2           mean_RNA/DNA_MCF10Ap53KO  \
enhancer_variant                    WT       mut                       WT   
0                             1.137255  0.733605                 1.228455   
1                             1.620464  1.152133                 1.645843   
2                             0.543610  0.348498                 0.551374   
3                             2.757653  1.644806                 2.971169   
6                             0.745280  0.652433                 0.608335   

                           RNA/DNA_HaCaT_1           RNA/DNA_HaCaT_2  \
enhancer_variant       mut              WT       mut              WT   
0                 0.646880        1.058830  0.638685        1.254867   
1                 1.356191        1.614002  0.918617        1.724374   
2                 0.365894        0.416229  0.338045        0.484690   
3                 1.757410        1.168821  1.835282        1.100346   
6                 0.668698        1.561466  1.715840        0.679508   

                           mean_RNA/DNA_HaCaT           RNA/DNA_SCC25_4  \
enhancer_variant       mut                 WT       mut              WT   
0                 0.879382           1.156849  0.759034        1.006967   
1                 1.219318           1.669188  1.068968        1.608028   
2                 0.366422           0.450459  0.352234        0.566084   
3                 1.115179           1.134583  1.475230        1.452863   
6                 0.939023           1.120487  1.327432        0.472392   

                           RNA/DNA_SCC25_2           mean_RNA/DNA_SCC25  \
enhancer_variant       mut              WT       mut                 WT   
0                 0.572140        1.058964  0.403181           1.032965   
1                 0.817092        0.402588  1.351475           1.005308   
2                 0.514911        0.841603  0.651545           0.703843   
3                 1.214646        2.005602  3.341386           1.729232   
6                 0.228029        0.485486  0.245629           0.478939   

                            
enhancer_variant       mut  
0                 0.487660  
1                 1.084283  
2                 0.583228  
3                 2.278016  
6                 0.236829

- Reformat data into a long format table 
- Each enhancer has appropriate variants and cell types paired

In [17]:
#use the wide table output to get a list of enhancers to keep (matched enhancers)
to_keep_mcf = wide_mcf['loc', ''].tolist()
to_keep_mhs = wide_mhs['loc', ''].tolist()
to_keep_tap63 = wide_tap63['loc', ''].tolist()
#filter
final_mcf = norm_mcf[norm_mcf['loc'].isin(to_keep_mcf)]
final_mhs = norm_mhs[(norm_mhs['loc'].isin(to_keep_mhs)) & (norm_mhs.enhancer_variant.isin(['WT', 'mut']))]
final_tap63 = tap63[(tap63['loc'].isin(to_keep_tap63)) & (tap63.enhancer_variant.isin(['WT', 'mut']))]
final_mhs

,loc,id,enhancer_variant,CPM_RNA_MCF10Ap53KO_1,CPM_RNA_MCF10Ap53KO_2,RNA/DNA_MCF10Ap53KO_1,RNA/DNA_MCF10Ap53KO_2,mean_RNA/DNA_MCF10Ap53KO,CPM_plasmid_1,CPM_RNA_HaCaT_1,CPM_RNA_HaCaT_2,RNA/DNA_HaCaT_1,RNA/DNA_HaCaT_2,mean_RNA/DNA_HaCaT,CPM_plasmid_2,CPM_RNA_SCC25_4,CPM_RNA_SCC25_2,RNA/DNA_SCC25_4,RNA/DNA_SCC25_2,mean_RNA/DNA_SCC25
0,chr1_3717092_3717116,chr1_3717092_3717116_WT,WT,10.014773,8.162671,0.650154,0.529916,0.590035,15.403700,14.418982,16.723220,0.974044,1.129702,1.051873,14.803212,14.257828,6.142934,0.925611,0.398796,0.662203
1,chr1_5652400_5652424,chr1_5652400_5652424_WT,WT,4.513400,7.805911,1.431832,2.476349,1.954091,3.152185,4.602268,3.858241,1.303404,1.092689,1.198047,3.530959,3.466828,5.790076,1.099817,1.836845,1.468331
3,chr1_31575754_31575773,chr1_31575754_31575773_WT,WT,10.448344,14.919708,1.704721,2.434255,2.069488,6.129065,6.635574,7.190358,1.003116,1.086984,1.045050,6.614962,8.137083,11.074924,1.327622,1.806952,1.567287
4,chr1_38027284_38027308,chr1_38027284_38027308_WT,WT,8.266275,7.862993,2.050158,1.950138,2.000148,4.032019,7.182162,7.703955,1.809575,1.941043,1.875309,3.968977,7.499506,6.824591,1.859988,1.692599,1.776294
5,chr1_38117813_38117832,chr1_38117813_38117832_WT,WT,36.874837,22.768430,2.798285,1.727806,2.263045,13.177656,9.095219,15.220009,0.704125,1.178288,0.941206,12.917054,10.703333,22.214005,0.812233,1.685733,1.248983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24637,chr9_135951083_135951107,chr9_135951083_135951107_mut,mut,2.693825,2.561538,0.085334,0.081143,0.083238,31.568159,1.913057,6.250851,0.064209,0.209801,0.137005,29.794146,15.532983,3.368188,0.492046,0.106696,0.299371
24638,chr9_136484776_136484795,chr9_136484776_136484795_mut,mut,19.098436,25.158723,1.214052,1.599293,1.406672,15.731157,9.740192,12.890032,0.574084,0.759735,0.666910,16.966483,17.963747,23.793846,1.141922,1.512530,1.327226
24639,chr9_136533879_136533893,chr9_136533879_136533893_mut,mut,13.973772,17.374217,0.936323,1.164172,1.050248,14.924092,13.970780,15.758659,0.912363,1.029121,0.970742,15.312743,12.783430,10.272975,0.856563,0.688348,0.772456
24640,chr9_137062571_137062595,chr9_137062571_137062595_mut,mut,19.702592,11.452000,3.405765,1.979578,2.692672,5.785070,16.113404,14.643778,1.566089,1.423253,1.494671,10.288947,8.511660,28.252686,1.471315,4.883724,3.177519


## Finalize tables
- Add log2(WT/mut) scores to measure p63RE-dependent enhancer activity
- Add categorical information: motif type, class, observation scores

In [18]:
#collect cell type information into the column
final_mcf_annot = final_mcf.melt(id_vars=['loc','enhancer_variant'],
									value_vars=['mean_RNA/DNA_MCF10A','mean_RNA/DNA_MCF10Ap53KO'],
									value_name='RNA/DNA', var_name='cell_line')
final_mhs_annot = final_mhs.melt(id_vars=['loc','enhancer_variant'],
									value_vars=['mean_RNA/DNA_MCF10Ap53KO','mean_RNA/DNA_HaCaT','mean_RNA/DNA_SCC25'],
									value_name='RNA/DNA', var_name='cell_line')
final_tap63_annot = final_tap63.melt(id_vars=['loc','enhancer_variant'],
									value_vars=['RNA/DNA_MCF10AGus','RNA/DNA_MCF10ATAp63B'],
									value_name='RNA/DNA', var_name='cell_line')
#simplify column names for the long table
replace_dict = {'mean_RNA/DNA_MCF10A':'MCF10A', 'mean_RNA/DNA_MCF10Ap53KO':'MCF10A p53KO','RNA/DNA_MCF10AGus':'GUS','RNA/DNA_MCF10ATAp63B':'TAp63B',
						 'mean_RNA/DNA_HaCaT':'HaCaT','mean_RNA/DNA_SCC25':'SCC25'}
final_mcf_annot.replace(replace_dict, inplace=True)
final_mhs_annot.replace(replace_dict, inplace=True)
final_tap63_annot.replace(replace_dict, inplace=True)
final_mcf_annot

,loc,enhancer_variant,cell_line,RNA/DNA
0,chr1_3717092_3717116,WT,MCF10A,0.704075
1,chr1_31575754_31575773,WT,MCF10A,1.586092
2,chr1_38117813_38117832,WT,MCF10A,1.681971
3,chr1_40886808_40886827,WT,MCF10A,2.157630
4,chr1_42941183_42941207,WT,MCF10A,1.995754
...,...,...,...,...
101285,chr9_132993510_132993534,fullShuffle,MCF10A p53KO,0.674189
101286,chr9_135016437_135016456,fullShuffle,MCF10A p53KO,1.347042
101287,chr9_136533879_136533893,fullShuffle,MCF10A p53KO,1.351870
101288,chr9_137062571_137062595,fullShuffle,MCF10A p53KO,1.612355


In [19]:
#collect enhancer variant information so it's easier to get WT/mut ratio
df_log2fc_mcf = final_mcf_annot[final_mcf_annot.enhancer_variant.isin(['WT','mut'])].pivot(index=['loc', 'cell_line'],
								columns=['enhancer_variant'], values=['RNA/DNA']).reset_index()
df_log2fc_mhs = final_mhs_annot[final_mhs_annot.enhancer_variant.isin(['WT','mut'])].pivot(index=['loc', 'cell_line'],
								columns=['enhancer_variant'], values=['RNA/DNA']).reset_index()
df_log2fc_tap63 = final_tap63_annot[final_tap63_annot.enhancer_variant.isin(['WT','mut'])].pivot(index=['loc', 'cell_line'],
								columns=['enhancer_variant'], values=['RNA/DNA']).reset_index()
#get log2 fold-change of WT/mut enhancer activity
df_log2fc_mcf['log2(WT/mut)'] = np.log2(df_log2fc_mcf[('RNA/DNA', 'WT')] / df_log2fc_mcf[('RNA/DNA', 'mut')])
df_log2fc_mhs['log2(WT/mut)'] = np.log2(df_log2fc_mhs[('RNA/DNA', 'WT')] / df_log2fc_mhs[('RNA/DNA', 'mut')])
df_log2fc_tap63['log2(WT/mut)'] = np.log2(df_log2fc_tap63[('RNA/DNA', 'WT')] / df_log2fc_tap63[('RNA/DNA', 'mut')])
#add categorical labels for FC cutoffs
fc_cutoff = 0.59 # ~ 1.5 FC
def dir(ratio):
	if ratio >= fc_cutoff: out = 'Activating'
	elif ratio <= -1*fc_cutoff: out = 'Repressing'
	else: out = 'Unchanged'
	return out
for tbl in [df_log2fc_mcf, df_log2fc_mhs, df_log2fc_tap63]:
	tbl.columns = tbl.columns.droplevel(level=1)
	tbl.columns = ['loc', 'cell_line', 'WT', 'mut', 'log2(WT/mut)'] #make all sure tables have this order
	tbl['activity'] = tbl['log2(WT/mut)'].apply(lambda x: dir(x))
#add activity column (log2(WT/mut)) to the normalized expression table
df_norm_long_1 = final_mcf_annot.merge(df_log2fc_mcf[['loc','cell_line','activity']], how='left', on=['loc', 'cell_line'])
df_norm_long_2 = final_mhs_annot.merge(df_log2fc_mhs[['loc','cell_line','activity']], how='left', on=['loc', 'cell_line'])
df_norm_long_3 = final_tap63_annot.merge(df_log2fc_tap63[['loc','cell_line','activity']], how='left', on=['loc', 'cell_line'])
df_log2fc_mhs

,loc,cell_line,WT,mut,log2(WT/mut),activity
0,chr10_100043539_100043558,HaCaT,1.156849,0.759034,0.607964,Activating
1,chr10_100043539_100043558,MCF10A p53KO,1.228455,0.646880,0.925274,Activating
2,chr10_100043539_100043558,SCC25,1.032965,0.487660,1.082843,Activating
3,chr10_100157341_100157365,HaCaT,1.669188,1.068968,0.642928,Activating
4,chr10_100157341_100157365,MCF10A p53KO,1.645843,1.356191,0.279266,Unchanged
...,...,...,...,...,...,...
29086,chr9_99116096_99116120,MCF10A p53KO,0.060792,0.123883,-1.027021,Repressing
29087,chr9_99116096_99116120,SCC25,0.130172,0.064725,1.008033,Activating
29088,chr9_99121705_99121724,HaCaT,1.115560,0.501759,1.152701,Activating
29089,chr9_99121705_99121724,MCF10A p53KO,0.533580,0.628994,-0.237343,Unchanged


In [20]:
#add categorical info about the enhancers from the table that was cleaned up earlier
df_info = pd.read_csv('p63enh_starrseq_info_long.csv', usecols=['obs_score','p63RE_class','strand','loc', 'enhancer_variant','p63RE_type','gc','Number of supporting p53 ChIP-seq datasets_p53'])
df_norm_long_1 = df_norm_long_1.merge(df_info, how='left', on=['loc', 'enhancer_variant'])
df_norm_long_2 = df_norm_long_2.merge(df_info, how='left', on=['loc', 'enhancer_variant'])
df_norm_long_3 = df_norm_long_3.merge(df_info, how='left', on=['loc', 'enhancer_variant'])
df_norm_long_1.rename(columns={'Number of supporting p53 ChIP-seq datasets_p53':'obs_p53'}, inplace=True)
df_norm_long_2.rename(columns={'Number of supporting p53 ChIP-seq datasets_p53':'obs_p53'}, inplace=True)
df_norm_long_3.rename(columns={'Number of supporting p53 ChIP-seq datasets_p53':'obs_p53'}, inplace=True)
# #remove irrelevant columns for merge onto fold-change table
df_info = df_info[df_info.enhancer_variant == 'WT']
df_info.drop(columns=['enhancer_variant','gc'], inplace=True)
df_log2fc_mcf = df_log2fc_mcf.merge(df_info, how='left', on='loc')
df_log2fc_mhs = df_log2fc_mhs.merge(df_info, how='left', on='loc')
df_log2fc_tap63 = df_log2fc_tap63.merge(df_info, how='left', on='loc')
df_log2fc_mcf.rename(columns={'Number of supporting p53 ChIP-seq datasets_p53':'obs_p53'}, inplace=True)
df_log2fc_mhs.rename(columns={'Number of supporting p53 ChIP-seq datasets_p53':'obs_p53'}, inplace=True)
df_log2fc_tap63.rename(columns={'Number of supporting p53 ChIP-seq datasets_p53':'obs_p53'}, inplace=True)
df_norm_long_3

,loc,enhancer_variant,cell_line,RNA/DNA,activity,obs_score,p63RE_class,strand,p63RE_type,gc,obs_p53
0,chr1_3717092_3717116,WT,GUS,0.565690,Unchanged,20,primary,+,Unique p63RE,48.333333,NaN
1,chr1_5652400_5652424,WT,GUS,1.387837,Unchanged,20,primary,+,Unique p63RE,53.333333,NaN
2,chr1_31575754_31575773,WT,GUS,1.236709,Unchanged,20,secondary,-,Unique p63RE,62.184874,NaN
3,chr1_38027284_38027308,WT,GUS,1.133709,Unchanged,20,primary,-,Unique p63RE,62.500000,NaN
4,chr1_38117813_38117832,WT,GUS,1.557596,Activating,20,primary,+,p53RE+p63RE,57.983193,11.0
...,...,...,...,...,...,...,...,...,...,...,...
54123,chr9_135951083_135951107,mut,TAp63B,0.049090,Repressing,8,primary,+,Unique p63RE,48.333333,NaN
54124,chr9_136484776_136484795,mut,TAp63B,0.473875,Unchanged,8,secondary,-,Unique p63RE,57.983193,NaN
54125,chr9_136533879_136533893,mut,TAp63B,1.007147,Unchanged,8,senary,-,Unique p63RE,59.166667,NaN
54126,chr9_137062571_137062595,mut,TAp63B,1.955652,Repressing,8,primary,-,Unique p63RE,55.833333,NaN


In [21]:
#normalized expression tables will be merged into one Table S2
df_norm_long_1.to_csv('p63enh_starrseq_normCounts_matched_MCF10A_WTp53KO_5variants.csv')
df_norm_long_2.to_csv('p63enh_starrseq_normCounts_matched_MCF10Ap53KO_HaCaT_SCC25_WTmut.csv')
df_norm_long_3.to_csv('p63enh_starrseq_normCounts_matched_MCF10A_GUSTAp63B_WTmut.csv')
df_log2fc_mcf.to_csv('p63enh_starrseq_log2fc_matched_MCF10A_WTp53KO_5variants.csv')
df_log2fc_mhs.to_csv('p63enh_starrseq_log2fc_matched_MCF10Ap53KO_HaCaT_SCC25_WTmut.csv')
df_log2fc_tap63.to_csv('p63enh_starrseq_log2fc_matched_MCF10A_GUSTAp63B_WTmut.csv')

## Prepare a table that has GC skew % accross the MPRA element for a selected window size (10 nt)

In [2]:
import os
import pandas as pd
os.chdir('..\data')
#use enhancers from matched MCf10A WT and p53KO table since it will be used to plot data in Fig. 4
df_pko = pd.read_excel('Table S3.xlsx', sheet_name='MCF10A_WT_p53KO')
df_pko = df_pko[(df_pko.cell_line == 'MCF10A p53KO') & (df_pko.enhancer_variant == 'WT')]
l = df_pko['loc'].tolist()
df_seqs = pd.read_csv('p63enh_starrseq_info_long.csv', usecols=['loc','seq','enhancer_variant'])
#we only need seqs from the WT and those that are matched in WT and p53KO
df_seqs = df_seqs[(df_seqs.enhancer_variant == 'WT') & (df_seqs['loc'].isin(l))]
#make loc:seq dictionary
enh = pd.Series(df_seqs.seq.values,index=df_seqs['loc']).to_dict()
#set rolling window size & function
window = 10
def gc_count(region):
	gc = ((region.count('G') + region.count('C')) / window) * 100
	return gc
d={}
for loc, seq in enh.items():
	s = seq.upper()
	pos=0
	d.update({loc:[]})
	while pos + window < len(s) + 1:
		skew = gc_count(s[pos:pos + window])
		d[loc].append(skew)
		pos += 1

In [3]:
#create a new df to add gc skew values for each enhancer sequence
df = pd.DataFrame()
for loc in d:
	temp = pd.DataFrame({'GC_skew': d[loc]})
	temp['loc'] = loc
	temp.reset_index(inplace=True)
	temp.rename(columns={'index':'position'}, inplace=True)
	df = pd.concat([df,temp], ignore_index=True)
#add 1 to offset python counting
df.position = df.position + 1
df = df.merge(df_pko, on='loc', how='left')
df.to_csv('p63enh_starrseq_gcskew_matched_MCF10Ap53KO.csv')
df
#each enhancer has a gc skew value (10nt window) for each position

,position,GC_skew,loc,enhancer_variant,cell_line,RNA/DNA,activity,obs_score,p63RE_class,strand,p63RE_type,gc,obs_p53
0,1,30.0,chr1_3717092_3717116,WT,MCF10A p53KO,0.590035,Unchanged,20,primary,+,Unique p63RE,48.333333,NaN
1,2,40.0,chr1_3717092_3717116,WT,MCF10A p53KO,0.590035,Unchanged,20,primary,+,Unique p63RE,48.333333,NaN
2,3,40.0,chr1_3717092_3717116,WT,MCF10A p53KO,0.590035,Unchanged,20,primary,+,Unique p63RE,48.333333,NaN
3,4,40.0,chr1_3717092_3717116,WT,MCF10A p53KO,0.590035,Unchanged,20,primary,+,Unique p63RE,48.333333,NaN
4,5,40.0,chr1_3717092_3717116,WT,MCF10A p53KO,0.590035,Unchanged,20,primary,+,Unique p63RE,48.333333,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1119586,107,60.0,chr9_137310751_137310775,WT,MCF10A p53KO,0.562648,Unchanged,8,primary,+,Unique p63RE,66.666667,NaN
1119587,108,60.0,chr9_137310751_137310775,WT,MCF10A p53KO,0.562648,Unchanged,8,primary,+,Unique p63RE,66.666667,NaN
1119588,109,70.0,chr9_137310751_137310775,WT,MCF10A p53KO,0.562648,Unchanged,8,primary,+,Unique p63RE,66.666667,NaN
1119589,110,60.0,chr9_137310751_137310775,WT,MCF10A p53KO,0.562648,Unchanged,8,primary,+,Unique p63RE,66.666667,NaN
